################################################################################
# Matthew Spitulnik ############################################################
# Natural Language Processing ##################################################
# Comparing Corpora with Corpus Statistics #####################################
# Project Summary: The goal of this project was to compare two different documents or sets of text-based data using corpus statistics. We were able to use data of our choosing, so I built Python data mining scripts to collect the Billboard top 100 songs for each year from 1980 through 2020, which was all converted into a data frame structure. I then built additional data mining scripts that used the list of collected song names to go out and download the lyrics for each song, which was then also converted into an easy-to-use data frame. Regex techniques were used to clean up lyric data and remove html formatting, punctuation, special characters, etc., and other data that would not contribute to any kind of analysis. The lyrics for songs written before the year 2000 were then compared to the lyrics for songs written during and after the year 2000 using NLP techniques like tokenization, stop word removal, lemmatizing, and n-gram analysis. I then created a report that communicated the results from the different analysis.
################################################################################

################################################################################
### Install and load required packages #########################################
################################################################################

In [ ]:
###Install the required libraries.
#%pip install pandas
#%pip install re
#%pip install requests
#%pip install html5lib
#%pip install nltk
#%pip install contractions
#%pip install lxml
#%pip install bs4

In [45]:
#Import the required libraries
import pandas as pd
import re
import requests
import html5lib
import nltk
#import sys
#!{sys.executable} -m pip install contractions
import contractions
from nltk import FreqDist
from lxml import html
from urllib import request
from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.collocations import *
nltk.download('stopwords', quiet=True)

True

################################################################################
### Import the data sets that will be used throughout the code #################
################################################################################

In [ ]:
###This creates a "default_directory" variable, where the directory path to the 
# data files folder containing all of the required data sets is saved so that it 
# does not need to be constantly re-entered. Remember to use forward slashes 
# instead of back slashes in the directory path. For example, if the datafiles 
# folder is saved in "C:\home\project\datafiles", then "C:/home/project" 
# would be inserted between the quotes below.
default_directory = "<DEFAULT DIRECTORY PATH HERE>"

In [25]:
#load the data frames that are created later so that the data mining scripts don't need to be run and waited for
songArtDF = pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv')
songArtDFOrg = pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDFOrg.csv')
songArtLyrics =pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtLyrics.csv')

################################################################################
### Collect and Clean Required Data ############################################
################################################################################

In [13]:
#start by running some tests to figure out how to collect the lists of top 100 billboard songs from each year.
testyearlink="https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1980"
testtables=pd.read_html(testyearlink)
testtables[0]

No.                                 Title                        Artist(s)
0     1                             "Call Me"                          Blondie
1     2  "Another Brick in the Wall, Part II"                       Pink Floyd
2     3                               "Magic"               Olivia Newton-John
3     4                       "Rock with You"                  Michael Jackson
4     5         "Do That to Me One More Time"               Captain & Tennille
..  ...                                   ...                              ...
95   96                          "In America"             Charlie Daniels Band
96   97                "Breakdown Dead Ahead"                       Boz Scaggs
97   98                               "Ships"                    Barry Manilow
98   99                      "All Night Long"                        Joe Walsh
99  100                             "Refugee"  Tom Petty and the Heartbreakers

[100 rows x 3 columns]

In [14]:
type(testtables[0])

pandas.core.frame.DataFrame

In [15]:
testtesttables=testtables[0][['Title','Artist(s)']]
testtesttables

Title                        Artist(s)
0                              "Call Me"                          Blondie
1   "Another Brick in the Wall, Part II"                       Pink Floyd
2                                "Magic"               Olivia Newton-John
3                        "Rock with You"                  Michael Jackson
4          "Do That to Me One More Time"               Captain & Tennille
..                                   ...                              ...
95                          "In America"             Charlie Daniels Band
96                "Breakdown Dead Ahead"                       Boz Scaggs
97                               "Ships"                    Barry Manilow
98                      "All Night Long"                        Joe Walsh
99                             "Refugee"  Tom Petty and the Heartbreakers

[100 rows x 2 columns]

In [16]:
testtable2=testtables[0]

In [17]:
testtable2['Year']=1980

In [18]:
testtable2

No.                                 Title  \
0     1                             "Call Me"   
1     2  "Another Brick in the Wall, Part II"   
2     3                               "Magic"   
3     4                       "Rock with You"   
4     5         "Do That to Me One More Time"   
..  ...                                   ...   
95   96                          "In America"   
96   97                "Breakdown Dead Ahead"   
97   98                               "Ships"   
98   99                      "All Night Long"   
99  100                             "Refugee"   

                          Artist(s)  Year  
0                           Blondie  1980  
1                        Pink Floyd  1980  
2                Olivia Newton-John  1980  
3                   Michael Jackson  1980  
4                Captain & Tennille  1980  
..                              ...   ...  
95             Charlie Daniels Band  1980  
96                       Boz Scaggs  1980  
97                    Barry Manilow  1980  
98                        Joe Walsh  1980  
99  Tom Petty and the Heartbreakers  1980  

[100 rows x 4 columns]

In [19]:
#now create the script that will go out to wikipedia for the top 100 billboard songs of each year from 1980-2019 and put them into a data frame
songArtDF=pd.DataFrame()
for i in range(1980,2020):
    songArtyear = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_" + str(i)
    temptables=pd.read_html(songArtyear)
    songArttable=temptables[0][['Title','Artist(s)']]
    songArttable['Year']=i
    
    CombDF=[songArtDF,songArttable]
    songArtDF=pd.concat(CombDF)
    
songArtDF=songArtDF.reset_index(drop=True)

KeyError: "None of [Index(['Title', 'Artist(s)'], dtype='object')] are in the [columns]"

In [ ]:
#####troubleshoot why this script is failing

In [20]:
songArtDF

Title  \
0                              "Call Me"   
1   "Another Brick in the Wall, Part II"   
2                                "Magic"   
3                        "Rock with You"   
4          "Do That to Me One More Time"   
..                                   ...   
95        "Are You Gonna Kiss Me or Not"   
96                              "Animal"   
97                     "You and Tequila"   
98                      "Colder Weather"   
99                             "My Last"   

                               Artist(s)  Year  
0                                Blondie  1980  
1                             Pink Floyd  1980  
2                     Olivia Newton-John  1980  
3                        Michael Jackson  1980  
4                     Captain & Tennille  1980  
..                                   ...   ...  
95                       Thompson Square  2011  
96                            Neon Trees  2011  
97  Kenny Chesney featuring Grace Potter  2011  
98                        Zac Brown Band  2011  
99        Big Sean featuring Chris Brown  2011  

[3200 rows x 3 columns]

In [21]:
i

2012

In [22]:
 temptables

[    0                                                  1
 0 NaN  This article relies excessively on references ...,
     No.                           Title                       Artist(s)
 0     1  "Somebody That I Used to Know"          Gotye featuring Kimbra
 1     2                 "Call Me Maybe"                Carly Rae Jepsen
 2     3                  "We Are Young"     Fun featuring Janelle Monáe
 3     4                      "Payphone"  Maroon 5 featuring Wiz Khalifa
 4     5                        "Lights"                  Ellie Goulding
 ..  ...                             ...                             ...
 95   96       "Somethin' 'Bout a Truck"                       Kip Moore
 96   97                         "Adorn"                          Miguel
 97   98               "Fly Over States"                    Jason Aldean
 98   99  "Even If It Breaks Your Heart"                  Eli Young Band
 99  100                  "Burn It Down"                     Linkin Park
 
 [100

In [23]:
#####appears that in 2012, a statement was added to the page that acted as a table, testing a way around this below
if 'Title' and 'Artist(s)' in temptables[0]:
    print('yes')
else:
    print('no')

if 'Title' and 'Artist(s)' in temptables[1]:
    print('yes')
else:
    print('no')

no
yes


In [24]:
#redo the script with above if statement to deal with pages that dont have the required table first
songArtDF=pd.DataFrame()
for i in range(1980,2020):
    songArtyear = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_" + str(i)
    temptables=pd.read_html(songArtyear)
    for h in range(0,len(temptables)):
        if 'Title' and 'Artist(s)' in temptables[h]:
            songArttable=temptables[h][['Title','Artist(s)']]
    songArttable['Year']=i
    
    CombDF=[songArtDF,songArttable]
    songArtDF=pd.concat(CombDF)
    
songArtDF=songArtDF.reset_index(drop=True)

In [ ]:
#create a column that establishes which document each song will be a part of: before the year 2000 or after it.
for i in songArtDF.index:
    if songArtDF.loc[i,'Year'] <2000:
        songArtDF.loc[i,'Before2000']='Yes'
    else:
        songArtDF.loc[i,'Before2000']='No'

In [ ]:
#change the Before2000 column to categorical: this may not be needed but most likely won't hurt
songArtDF=songArtDF.astype({'Before2000':'category'})

In [26]:
songArtDF

Title                       Artist(s)  \
0                                "Call Me"                         Blondie   
1     "Another Brick in the Wall, Part II"                      Pink Floyd   
2                                  "Magic"              Olivia Newton-John   
3                          "Rock with You"                 Michael Jackson   
4            "Do That to Me One More Time"              Captain & Tennille   
...                                    ...                             ...   
3995                         "Eyes on You"                      Chase Rice   
3996                       "All to Myself"                      Dan + Shay   
3997                           "Boyfriend"  Ariana Grande and Social House   
3998                        "Walk Me Home"                            Pink   
3999                             "Robbery"                      Juice Wrld   

      Year Before2000                                             Lyrics  
0     1980        Yes  ['\nColour me your colour, baby', 'Colour me y...  
1     1980        Yes  ["\nWe don't need no education ", "We don't ne...  
2     1980        Yes  ['\nCome take my hand', '\nYou should know me'...  
3     1980        Yes  ['\n[1st Verse]', '\nGirl, Close Your Eyes', '...  
4     1980        Yes  ['\nDo that to me one more time', '\nOnce is n...  
...    ...        ...                                                ...  
3995  2019         No  ["We've been to both Carolina's", '\nSeen a bi...  
3996  2019         No  ["I'm jealous of the blue jeans that you're we...  
3997  2019         No  ['\n(Feat. Social House)', '\n', '\n[Ariana Gr...  
3998  2019         No                                                 []  
3999  2019         No                                                 []  

[4000 rows x 5 columns]

In [ ]:
###exporting this dataframe so it doesn't need to be created every time
#songArtDF.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv',index=False)

In [27]:
###now test how to use the data frame to collect each songs lyrics
#azlyrics.com uses the name of the artist with no spaces/punctuation/special characters, so remove those
spaceremovetest= re.sub('[\W]','',songArtDF.loc[3997,'Artist(s)'])
spaceremovetest

'ArianaGrandeandSocialHouse'

In [28]:
songArtDF.loc[3998,'Title']

'"Walk Me Home"'

In [29]:
songArtDF.loc[3342,'Artist(s)']

'Macklemore & Ryan Lewis featuring Mary Lambert'

In [30]:
#same thing for the title of the song
quoteremovetest= re.sub('[\W]','',songArtDF.loc[3997,'Title'])
quoteremovetest

'Boyfriend'

In [31]:
#if a song is performed by an artist, with another artist featured, on azlyrics.com only the primary artist is used. Next few cells will be an if statment to try to deal with this.
lyricURLtest='https://www.azlyrics.com/lyrics/' + spaceremovetest + '/' + quoteremovetest + '.html'
lyricURLtest

'https://www.azlyrics.com/lyrics/ArianaGrandeandSocialHouse/Boyfriend.html'

In [32]:
#this tests if the initial guess at the song lyrics URL works, and if it doesn't what to do about it
get = requests.get(lyricURLtest)
if get.status_code == 200:
    print('This worked')
else:
    print('This did not work')

This did not work


In [33]:
get = requests.get(lyricURLtest)
if get.status_code == 200:
    print('This worked')
else:
    URLandMatchTest=re.search(' and ', songArtDF.loc[3997,'Artist(s)'])
    if URLandMatchTest:
        print('This song has and in it')

This song has and in it


In [34]:
#now figure out how to remove everything after ' and ' or ' & '
re.findall(' and (?<= and ).*$',songArtDF.loc[3997,'Artist(s)'])

[' and Social House']

In [35]:
if ' and ' in songArtDF.loc[3997,'Artist(s)']:
    print('Yes')

Yes


In [36]:
#try one more
re.findall(' & (?<= & ).*$',songArtDF.loc[3342,'Artist(s)'])

[' & Ryan Lewis featuring Mary Lambert']

In [37]:
#and one more
re.findall(' featuring (?<= featuring ).*$',songArtDF.loc[3342,'Artist(s)'])

[' featuring Mary Lambert']

In [38]:
#now try doing this with a re.sub to remove everything after those characters
andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[3997,'Artist(s)'])
andsubremoval

'Ariana Grande'

In [39]:
ampsubremoval=re.sub(' & (?<= & ).*$','',songArtDF.loc[3342,'Artist(s)'])
ampsubremoval

'Macklemore'

In [40]:
featsubremoval=re.sub(' featuring (?<= featuring ).*$','',songArtDF.loc[3342,'Artist(s)'])
featsubremoval

'Macklemore & Ryan Lewis'

In [41]:
#now get the lyrics from a working URL using the xpatch, or the breakdown of where in the structure of the html the text is located.
#/html/body/div[2]/div[2]/div[2]/div[5]
#/html/body/div[2]/div[2]/div[2]/div[5]
#/html/body/div[2]/div[2]/div[2]/div[5]
templyricsURL='https://www.azlyrics.com/lyrics/barbrastreisand/nomoretearsenoughisenough.html'
templyricshtml=requests.get(templyricsURL)
templyricscontent=html.fromstring(templyricshtml.content)
templyrics=templyricscontent.xpath('/html/body/div[2]/div[2]/div[2]/div[5]/text()')
print(templyrics[5])
templyrics


No sunshine


['\r\n',
 "\r\nIt's raining, it's pouring",
 '\nMy love life is boring me to tears',
 '\nAfter all these years',
 '\n',
 '\nNo sunshine',
 '\nNo moonlight',
 '\nNo stardust',
 '\nNo sign of romance',
 "\nWe don't stand a chance",
 '\n',
 "\nI've always dreamed",
 '\nI found the perfect lover',
 '\nBut he turned out to be',
 '\nLike every other man',
 "\nI've loved, I've loved",
 '\n',
 '\nRaining (raining)',
 '\nPouring (pouring)',
 "\nThere's nothing left for us here",
 "\nAnd we won't waste another tear",
 '\n',
 "\nIf you've had enough",
 "\nDon't put up with his stuff",
 "\nDon't you do it",
 '\n',
 "\nIf you've had your fill",
 '\nGet the check pay the bill',
 '\nYou can do it',
 '\n',
 '\nTell him to just get out',
 '\n',
 '\nNothing left to talk about',
 '\n',
 '\nPack his raincoat show him out',
 '\n',
 '\nJust look him in the eye',
 '\nAnd simply shout',
 '\nEnough is enough',
 "\nI can't go on",
 "\nI can't go on",
 '\nNo more no',
 '\nEnough is enough',
 '\nI want him out',


In [ ]:
#now create initial script that will collect all of the lyrics
for i in songArtDF.index:
    artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
    titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
    templyricURL='https://www.azlyrics.com/lyrics/' + artnospace + '/' + titlenospace + '.html'
    templyricURL=templyricURL.lower()
    
    linktest = requests.get(templyricURL)
    if linktest.status_code == 200:
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('/html/body/div[2]/div[2]/div[2]/div[5]/text()'))
        songArtDF.loc[i,'Lyrics']=templyric
    else:
        if ' and ' in songArtDF.loc[i,'Artist(s)']:
            andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
        else:
            andsubremoval=songArtDF.loc[i,'Artist(s)']
        if ' & ' in andsubremoval:
            ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
        else:
            ampsubremoval=andsubremoval
        if ' featuring ' in ampsubremoval:
            featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
        else:
            featsubremoval=ampsubremoval
        artnospace= re.sub('[\W]','',featsubremoval)
        templyricURL='https://www.azlyrics.com/lyrics/' + artnospace + '/' + titlenospace + '.html'
        templyricURL=templyricURL.lower()
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('/html/body/div[2]/div[2]/div[2]/div[5]/text()'))
        songArtDF.loc[i,'Lyrics']=templyric

In [ ]:
songArtDF.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv',index=False)

In [ ]:
songArtDF

In [ ]:
#azlyrics.com started failing after 8 searches, trying songlyrics.com instead
for i in songArtDF.index:
    artnospace=songArtDF.loc[i,'Artist(s)'].replace(',','')
    artnospace= re.sub('[\W]','-',artnospace)
    titlenospace=songArtDF.loc[i,'Title'].replace(',','')
    titlenospace= re.sub('[\W]','-',titlenospace)
    templyricURL='https://www.songlyrics.com/' + artnospace + '/' + titlenospace + '/'
    templyricURL=templyricURL.lower()
    
    linktest = requests.get(templyricURL)
    if linktest.status_code == 200:
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('//*[@id="songLyricsDiv"]'))
        songArtDF.loc[i,'Lyrics']=templyric
    else:
        if ' and ' in songArtDF.loc[i,'Artist(s)']:
            andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
        else:
            andsubremoval=songArtDF.loc[i,'Artist(s)']
        if ' & ' in andsubremoval:
            ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
        else:
            ampsubremoval=andsubremoval
        if ' featuring ' in ampsubremoval:
            featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
        else:
            featsubremoval=ampsubremoval
        artnospace=featsubremoval.replace(',','')
        artnospace= re.sub('[\W]','-',artnospace)
        templyricURL='https://www.songlyrics.com/' + artnospace + '/' + titlenospace + '/'
        templyricURL=templyricURL.lower()
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('//*[@id="songLyricsDiv"]'))
        songArtDF.loc[i,'Lyrics']=templyric

In [42]:
#the structure of songlyrics.com ULRs gave me some trouble, may need to try another site again
testpolice=re.sub('^The ','',songArtDF.loc[156,'Artist(s)'])

In [43]:
firstletter=testpolice[0].lower()
print(type(firstletter))
firstletter

<class 'str'>


'p'

In [44]:
thetest = re.match('^The ', songArtDF.loc[156,'Artist(s)'])
if thetest:
    print(True)
else:
    print(False)

True


In [ ]:
#had issues with songlyrics.com also, trying one more site, lyricsondemand.com
for i in songArtDF.index:
    theremoval=re.sub('^The ','',songArtDF.loc[i,'Artist(s)'])
    firstletter=theremoval[0].lower()
    artnospace= re.sub('[\W]','',theremoval)
    titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
    templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
    templyricURL=templyricURL.lower()
    
    linktest = requests.get(templyricURL)
    if linktest.status_code == 200:
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[3]/text()'))
        songArtDF.loc[i,'Lyrics']=templyric
    else:
        thetest = re.match('^The ', songArtDF.loc[i,'Artist(s)'])
        if thetest:
            artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
            templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
            templyricURL=templyricURL.lower()
    
            linktest = requests.get(templyricURL)
            if linktest.status_code == 200:
                templyrichtml=requests.get(templyricURL)
                templyriccontent=html.fromstring(templyrichtml.content)
                templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[3]/text()'))
                songArtDF.loc[i,'Lyrics']=templyric
            else:
                if ' and ' in songArtDF.loc[i,'Artist(s)']:
                    andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                else:
                    andsubremoval=songArtDF.loc[i,'Artist(s)']
                if ' & ' in andsubremoval:
                    ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                else:
                    ampsubremoval=andsubremoval
                if ' featuring ' in ampsubremoval:
                    featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                else:
                    featsubremoval=ampsubremoval
                    artnospace= re.sub('[\W]','',featsubremoval)
        else:
            if ' and ' in songArtDF.loc[i,'Artist(s)']:
                andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
            else:
                andsubremoval=songArtDF.loc[i,'Artist(s)']
            if ' & ' in andsubremoval:
                ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
            else:
                ampsubremoval=andsubremoval
            if ' featuring ' in ampsubremoval:
                featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
            else:
                featsubremoval=ampsubremoval
                artnospace= re.sub('[\W]','',featsubremoval)
        templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
        templyricURL=templyricURL.lower()
        templyrichtml=requests.get(templyricURL)
        templyriccontent=html.fromstring(templyrichtml.content)
        templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[3]/text()'))
        songArtDF.loc[i,'Lyrics']=templyric

In [ ]:
songArtDF.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv',index=False)

In [ ]:
###some songs were collected, but a large number did not work, troubleshooting again

In [ ]:
templyricURL

In [ ]:
linktest = requests.get(templyricURL)
if linktest.status_code == 200:
    print('TRUE')

In [ ]:
if thetest:
    print('TRUE')

In [ ]:
print(artnospace)
print(songArtDF.loc[i,'Artist(s)'])
andsubremoval

In [ ]:
templyricsSTR=str(templyrics)
print(templyricsSTR)
templyricsSTR

In [ ]:
templyricsSTRsoup = BeautifulSoup(templyricsSTR, 'html.parser')
print(type(templyricsSTRsoup))
templyricsSTRsoup

In [ ]:
#looking at another link
templyricsURL='https://www.azlyrics.com/lyrics/macklemore/samelove.html'
templyricshtml=requests.get(templyricsURL)
templyricscontent=html.fromstring(templyricshtml.content)
templyrics=templyricscontent.xpath('/html/body/div[2]/div[2]/div[2]/div[5]/text()')
print(templyrics)
templyricsSTR=str(templyrics)
templyricsSTR

In [ ]:
for i in songArtDF.index:
    if songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']':
        print(songArtDF.loc[i,'Lyrics'], ' || ','//*[@id="sbmtlyr"]')

In [ ]:
for i in songArtDF.index:
    if songArtDF.loc[i,'Lyrics']=='[]':
        print(songArtDF.loc[i,'Lyrics'], ' || ','//*[@id="ldata"]/div[4]')

In [ ]:
#a large number of songs didn't work, the xpath to the song lyric text appears to be different for many of those songs, adjusting script to fill in the lyrics that didn't work
for i in songArtDF.index:
    if songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']':
        theremoval=re.sub('^The ','',songArtDF.loc[i,'Artist(s)'])
        firstletter=theremoval[0].lower()
        artnospace= re.sub('[\W]','',theremoval)
        titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
        templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
        templyricURL=templyricURL.lower()
    
        linktest = requests.get(templyricURL)
        if linktest.status_code == 200:
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
            songArtDF.loc[i,'Lyrics']=templyric
        else:
            thetest = re.match('^The ', songArtDF.loc[i,'Artist(s)'])
            if thetest:
                artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
                templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                templyricURL=templyricURL.lower()
    
                linktest = requests.get(templyricURL)
                if linktest.status_code == 200:
                    templyrichtml=requests.get(templyricURL)
                    templyriccontent=html.fromstring(templyrichtml.content)
                    templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
                    songArtDF.loc[i,'Lyrics']=templyric
                else:
                    if ' and ' in songArtDF.loc[i,'Artist(s)']:
                        andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                    else:
                        andsubremoval=songArtDF.loc[i,'Artist(s)']
                    if ' & ' in andsubremoval:
                        ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                    else:
                        ampsubremoval=andsubremoval
                    if ' featuring ' in ampsubremoval:
                        featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                    else:
                        featsubremoval=ampsubremoval
                    artnospace= re.sub('[\W]','',featsubremoval)
            else:
                if ' and ' in songArtDF.loc[i,'Artist(s)']:
                    andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                else:
                    andsubremoval=songArtDF.loc[i,'Artist(s)']
                if ' & ' in andsubremoval:
                    ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                else:
                    ampsubremoval=andsubremoval
                if ' featuring ' in ampsubremoval:
                    featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                else:
                    featsubremoval=ampsubremoval
                artnospace= re.sub('[\W]','',featsubremoval)
            templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
            templyricURL=templyricURL.lower()
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
            songArtDF.loc[i,'Lyrics']=templyric
    else:
        if songArtDF.loc[i,'Lyrics']=='[]':
            theremoval=re.sub('^The ','',songArtDF.loc[i,'Artist(s)'])
            firstletter=theremoval[0].lower()
            artnospace= re.sub('[\W]','',theremoval)
            titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
            templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
            templyricURL=templyricURL.lower()
    
            linktest = requests.get(templyricURL)
            if linktest.status_code == 200:
                templyrichtml=requests.get(templyricURL)
                templyriccontent=html.fromstring(templyrichtml.content)
                templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
                songArtDF.loc[i,'Lyrics']=templyric
            else:
                thetest = re.match('^The ', songArtDF.loc[i,'Artist(s)'])
                if thetest:
                    artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
                    templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                    templyricURL=templyricURL.lower()
    
                    linktest = requests.get(templyricURL)
                    if linktest.status_code == 200:
                        templyrichtml=requests.get(templyricURL)
                        templyriccontent=html.fromstring(templyrichtml.content)
                        templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
                        songArtDF.loc[i,'Lyrics']=templyric
                    else:
                        if ' and ' in songArtDF.loc[i,'Artist(s)']:
                            andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                        else:
                            andsubremoval=songArtDF.loc[i,'Artist(s)']
                        if ' & ' in andsubremoval:
                            ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                        else:
                            ampsubremoval=andsubremoval
                        if ' featuring ' in ampsubremoval:
                            featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                        else:
                            featsubremoval=ampsubremoval
                        artnospace= re.sub('[\W]','',featsubremoval)
                else:
                    if ' and ' in songArtDF.loc[i,'Artist(s)']:
                        andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                    else:
                        andsubremoval=songArtDF.loc[i,'Artist(s)']
                    if ' & ' in andsubremoval:
                        ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                    else:
                        ampsubremoval=andsubremoval
                    if ' featuring ' in ampsubremoval:
                        featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                    else:
                        featsubremoval=ampsubremoval
                    artnospace= re.sub('[\W]','',featsubremoval)
                templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                templyricURL=templyricURL.lower()
                templyrichtml=requests.get(templyricURL)
                templyriccontent=html.fromstring(templyrichtml.content)
                templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
                songArtDF.loc[i,'Lyrics']=templyric

In [ ]:
print(templyriccontent)
templyric

In [ ]:
songArtDF.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv',index=False)

In [ ]:
songArtDF

In [ ]:
templyricURL='https://www.lyricsondemand.com/n/nickjonaslyrics/jealouslyrics.html'
templyricURL=templyricURL.lower()
templyrichtml=requests.get(templyricURL)
templyriccontent=html.fromstring(templyrichtml.content)
templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
templyric

In [ ]:
#still didn't get a bunch of songs, realized the syntax for some of the if statements were incorrect, corrected those
for i in range(3921,3922):
    if songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']':
        theremoval=re.sub('^The ','',songArtDF.loc[i,'Artist(s)'])
        firstletter=theremoval[0].lower()
        artnospace= re.sub('[\W]','',theremoval)
        titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
        templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
        templyricURL=templyricURL.lower()
    
        linktest = requests.get(templyricURL)
        if linktest.status_code == 200:
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
            #songArtDF.loc[i,'Lyrics']=templyric
            print(1)
        else:
            thetest = re.match('^The ', songArtDF.loc[i,'Artist(s)'])
            if thetest:
                artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
                templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                templyricURL=templyricURL.lower()
    
                linktest = requests.get(templyricURL)
                if linktest.status_code == 200:
                    templyrichtml=requests.get(templyricURL)
                    templyriccontent=html.fromstring(templyrichtml.content)
                    templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
                    #songArtDF.loc[i,'Lyrics']=templyric
                    print(2)
                else:
                    if ' and ' in songArtDF.loc[i,'Artist(s)']:
                        andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                    else:
                        andsubremoval=songArtDF.loc[i,'Artist(s)']
                    if ' & ' in andsubremoval:
                        ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                    else:
                        ampsubremoval=andsubremoval
                    if ' featuring ' in ampsubremoval:
                        featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                    else:
                        featsubremoval=ampsubremoval
                    artnospace= re.sub('[\W]','',featsubremoval)
            else:
                if ' and ' in songArtDF.loc[i,'Artist(s)']:
                    andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                else:
                    andsubremoval=songArtDF.loc[i,'Artist(s)']
                if ' & ' in andsubremoval:
                    ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                else:
                    ampsubremoval=andsubremoval
                if ' featuring ' in ampsubremoval:
                    featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                else:
                    featsubremoval=ampsubremoval
                artnospace= re.sub('[\W]','',featsubremoval)
            templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
            templyricURL=templyricURL.lower()
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            #templyric=str(templyriccontent.xpath('//*[@id="sbmtlyr"]/text()'))
            templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
            #songArtDF.loc[i,'Lyrics']=templyric
            print(3)
    else:
        if songArtDF.loc[i,'Lyrics']=='[]':
            theremoval=re.sub('^The ','',songArtDF.loc[i,'Artist(s)'])
            firstletter=theremoval[0].lower()
            artnospace= re.sub('[\W]','',theremoval)
            titlenospace= re.sub('[\W]','',songArtDF.loc[i,'Title'])
            templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
            templyricURL=templyricURL.lower()
    
            linktest = requests.get(templyricURL)
            if linktest.status_code == 200:
                templyrichtml=requests.get(templyricURL)
                templyriccontent=html.fromstring(templyrichtml.content)
                templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
                #songArtDF.loc[i,'Lyrics']=templyric
                print(4)
            else:
                thetest = re.match('^The ', songArtDF.loc[i,'Artist(s)'])
                if thetest:
                    artnospace= re.sub('[\W]','',songArtDF.loc[i,'Artist(s)'])
                    templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                    templyricURL=templyricURL.lower()
    
                    linktest = requests.get(templyricURL)
                    if linktest.status_code == 200:
                        templyrichtml=requests.get(templyricURL)
                        templyriccontent=html.fromstring(templyrichtml.content)
                        templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[5]/text()'))
                        #songArtDF.loc[i,'Lyrics']=templyric
                        print(5)
                    else:
                        if ' and ' in songArtDF.loc[i,'Artist(s)']:
                            andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                        else:
                            andsubremoval=songArtDF.loc[i,'Artist(s)']
                        if ' & ' in andsubremoval:
                            ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                        else:
                            ampsubremoval=andsubremoval
                        if ' featuring ' in ampsubremoval:
                            featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                        else:
                            featsubremoval=ampsubremoval
                        artnospace= re.sub('[\W]','',featsubremoval)
                else:
                    if ' and ' in songArtDF.loc[i,'Artist(s)']:
                        andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
                    else:
                        andsubremoval=songArtDF.loc[i,'Artist(s)']
                    if ' & ' in andsubremoval:
                        ampsubremoval=re.sub(' & (?<= & ).*$','',andsubremoval)
                    else:
                        ampsubremoval=andsubremoval
                    if ' featuring ' in ampsubremoval:
                        featsubremoval=re.sub(' featuring (?<= featuring ).*$','',ampsubremoval)
                    else:
                        featsubremoval=ampsubremoval
                    artnospace= re.sub('[\W]','',featsubremoval)
                templyricURL='https://www.lyricsondemand.com/' + firstletter + '/' + artnospace + 'lyrics/' + titlenospace + 'lyrics.html'
                templyricURL=templyricURL.lower()
                templyrichtml=requests.get(templyricURL)
                templyriccontent=html.fromstring(templyrichtml.content)
                templyric=str(templyriccontent.xpath('//*[@id="ldata"]/div[4]/text()'))
                #songArtDF.loc[i,'Lyrics']=templyric
                print(6)

In [ ]:
###still missing around 750 songs, more troubleshooting

In [ ]:
templyricURL

In [ ]:
featsubremoval

In [ ]:
artnospace

In [ ]:
titlenospace

In [ ]:
templyric

In [ ]:
songArtDF.loc[30,'Lyrics']='[\'\\n\', \'\\n\']'

In [ ]:
songArtDF.loc[i,'Lyrics']

In [ ]:
print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[]'])

In [ ]:
print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[]'])

In [ ]:
req = Request(
    url='https://www.lyricsondemand.com/m/maroon5lyrics/girlslikeyoulyrics.html', 
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()
type(webpage)

In [ ]:
testyear = "https://www.lyricsondemand.com/k/kennyrogerslyrics/dontfallinlovewithadreamerlyrics.html"
#testResponse =request.urlopen(testyear)
#testHTML = testResponse.read().decode('utf8')
#testsoup = BeautifulSoup(testHTML, 'html.parser')
testsoup = BeautifulSoup(webpage, 'html.parser')
testsoupCLASS=testsoup.find_all('div', class_='lcontent')
testsoupCLASS

In [ ]:
testsoup.find('div', class_='lcontent').text

In [ ]:
webpage

In [ ]:
webpage = requests.get('https://www.lyricsondemand.com/m/maroon5lyrics/girlslikeyoulyrics.html',headers={'User-Agent': 'Mozilla/5.0'})
testsoup = BeautifulSoup(webpage.text, 'lxml')
testsoup.body.get_text(' ', strip=True)

In [ ]:
for i in songArtDF.index:
    if songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\\n\']' or songArtDF.loc[i,'Lyrics']=='[\'\\n\', \'\\n\']' or songArtDF.loc[i,'Lyrics']=='[]' or songArtDF.loc[i,'Lyrics']=='[\'\\n\']':
        songArtDF.loc[i,'Lyrics']='[]'

In [ ]:
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\']'])
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[]'])
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\']'])

In [ ]:
print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[]'])
print(songArtDF['Lyrics'].value_counts()['[\'\\n\']'])

In [ ]:
templyricURL

In [ ]:
songArtDF

In [ ]:
#determined that the structure of some pages from lyricsondemand.com did not allow for the song lyric tests to be downloaded, going back to songlyrics.com to try to get as many remaining songs as possible
for i in songArtDF.index:
#for i in range(3979,3980):
    if songArtDF.loc[i,'Lyrics']=='[]':
        artnospace=songArtDF.loc[i,'Artist(s)'].replace(',','')
        artnospace= re.sub('[\W]','-',artnospace)
        titlenospace=re.sub('^"','',songArtDF.loc[i,'Title'])
        titlenospace=titlenospace.replace(',','')
        titlenospace= re.sub('[\W]','-',titlenospace)
        templyricURL='https://www.songlyrics.com/' + artnospace + '/' + titlenospace + 'lyrics/'
        templyricURL=templyricURL.lower()
    
        linktest = requests.get(templyricURL)
        if linktest.status_code == 200:
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            templyric=str(templyriccontent.xpath('//*[@id="songLyricsDiv"]/text()'))
            songArtDF.loc[i,'Lyrics']=templyric
            #print(1)

In [ ]:
songArtDF.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv',index=False)

In [ ]:
for i in songArtDF.index:
    if 'We do not have the lyrics for' in songArtDF.loc[i,'Lyrics']:
        songArtDF.loc[i,'Lyrics']='[]'

In [ ]:
for i in songArtDF.index:
#for i in range(3925,3926):
    if songArtDF.loc[i,'Lyrics']=='[]':
        if ' and ' in songArtDF.loc[i,'Artist(s)']:
            featart=re.findall('(?<= and ).*$', songArtDF.loc[i,'Artist(s)'])
            featart=str(featart[0])
            andsubremoval=re.sub(' and (?<= and ).*$','',songArtDF.loc[i,'Artist(s)'])
            featadd=andsubremoval+'-feat-'+featart
        elif ' & ' in songArtDF.loc[i,'Artist(s)']:
            featart=re.findall('(?<= & ).*$', songArtDF.loc[i,'Artist(s)'])
            featart=str(featart[0])
            ampsubremoval=re.sub(' & (?<= & ).*$','',songArtDF.loc[i,'Artist(s)'])
            featadd=ampsubremoval+'-feat-'+featart
        elif ' featuring ' in songArtDF.loc[i,'Artist(s)']:
            featart=re.findall('(?<= featuring ).*$', songArtDF.loc[i,'Artist(s)'])
            featart=str(featart[0])
            featsubremoval=re.sub(' featuring (?<= featuring ).*$','',songArtDF.loc[i,'Artist(s)'])
            featadd=featsubremoval+'-feat-'+featart
        else:
            featadd=songArtDF.loc[i,'Artist(s)']
        artnospace=featadd.replace(',','')
        artnospace= re.sub('[\W]','-',artnospace)
        titlenospace=re.sub('^"','',songArtDF.loc[i,'Title'])
        titlenospace=titlenospace.replace(',','')
        titlenospace= re.sub('[\W]','-',titlenospace)
        templyricURL='https://www.songlyrics.com/' + artnospace + '/' + titlenospace + 'lyrics/'
        templyricURL=templyricURL.lower()
        
        linktest = requests.get(templyricURL)
        if linktest.status_code == 200:
            templyrichtml=requests.get(templyricURL)
            templyriccontent=html.fromstring(templyrichtml.content)
            templyric=str(templyriccontent.xpath('//*[@id="songLyricsDiv"]/text()'))
            songArtDF.loc[i,'Lyrics']=templyric

In [ ]:
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\']'])
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\', \'\\n\\n\']'])
print(songArtDF['Lyrics'].value_counts()['[]'])
#print(songArtDF['Lyrics'].value_counts()['[\'\\n\']'])

In [50]:
for i in songArtDF.index:
    if 'We do not have the lyrics for' in songArtDF.loc[i,'Lyrics']:
        songArtDF.loc[i,'Lyrics']='[]'

In [ ]:
titlenospace

In [ ]:
templyric

In [ ]:
templyricURL

In [52]:
songArtDF.loc[23,'Title']

'"Yes, I\'m Ready"'

In [51]:
songArtDF.loc[23,'Title'].replace(',','')

'"Yes I\'m Ready"'

In [ ]:
re.sub('[\W]','-',titlenospace)

In [ ]:
re.findall('(?<= featuring ).*$', songArtDF.loc[3917,'Artist(s)'])

In [ ]:
re.findall('(?<= and ).*$', songArtDF.loc[3925,'Artist(s)'])

In [ ]:
########Able to get the list down to only 200 song lyrics missing, going to do cleanup of the lyrics now
#IMPORTANT NOTE# Because it took so many intermittent steps to get as many of the song lyrics as possible
# collected, the final result of this work was exported so that it could then just be imported without having
# to wait for all of the above data mining scripts to finish. There is code to import the resulting file at the
# top of the page already, and the name of that file is songArtDF.

In [ ]:
#make a copy of song list as is before editting
songArtDFOrg=songArtDF.copy(deep=True)
songArtDFOrg.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDFOrg.csv',index=False)

In [ ]:
#remove songs with no lyrics and reset the index
songArtLyrics=songArtDF[songArtDF['Lyrics'] != '[]']
songArtLyrics=songArtLyrics.reset_index(drop=True)

In [ ]:
songArtLyrics.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtLyrics.csv',index=False)

In [ ]:
#figure out how many songs we found lyrics for before and after 2000
print(songArtLyrics['Before2000'].value_counts()['Yes']) #1951 songs before 2000
print(songArtLyrics['Before2000'].value_counts()['No']) #1849 songs after 2000

In [ ]:
songArtLyrics

In [ ]:
#remove line breaks
for i in songArtLyrics.index:
    songArtLyrics.loc[i,'Lyrics']=songArtLyrics.loc[i,'Lyrics'].replace('\\n','')
    songArtLyrics.loc[i,'Lyrics']=songArtLyrics.loc[i,'Lyrics'].replace('\\r','')
    songArtLyrics.loc[i,'Lyrics']=songArtLyrics.loc[i,'Lyrics'].replace('\\','')

In [ ]:
#remove contractions
for i in songArtLyrics.index:
    songArtLyrics.loc[i,'Lyrics']=contractions.fix(songArtLyrics.loc[i,'Lyrics'])

In [ ]:
#everything lowercase:
for i in songArtLyrics.index:
    songArtLyrics.loc[i,'Lyrics']=songArtLyrics.loc[i,'Lyrics'].lower()

In [ ]:
#remove punctuation
for i in songArtLyrics.index:
    songArtLyrics.loc[i,'Lyrics']=re.sub(r'[^\w\s]','',songArtLyrics.loc[i,'Lyrics'])

In [ ]:
songArtLyrics.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtLyrics.csv',index=False)

################################################################################
### Perform Analysis of Data ###################################################
################################################################################

In [ ]:
#turn all of the lyrics into two different corpus's, one of the songs before 2000, one of the lyrics from after 2000
#first before 2000
'''before2000corp=[]
after2000corp=[]
for i in songArtLyrics.index:
    titlenospace= re.sub('[\W]','',songArtLyrics.loc[i,'Title'])
    locals()[titlenospace]=songArtLyrics.loc[i,'Lyrics']
    if songArtLyrics.loc[i,'Before2000']=='Yes':
        before2000corp.append(titlenospace)
    else:
        before2000corp.append(titlenospace)'''

In [3]:
#instead of a corpus, combine all the lyrics into two lists of lyrics, before 2000 and after 2000. The order of the lyrics will not change, so bigram measurements can still be done later
before2000lyr=[]
after2000lyr=[]
for i in songArtLyrics.index:
    if songArtLyrics.loc[i,'Before2000']=='Yes':
        before2000lyr.append(songArtLyrics.loc[i,'Lyrics'])
    else:
        after2000lyr.append(songArtLyrics.loc[i,'Lyrics'])

In [4]:
#tokenize the before 2000 words
before2000TOK=nltk.word_tokenize(str(before2000lyr))

In [5]:
print(len(before2000TOK))
len(before2000TOK)/1951

536317


274.8933880061507

In [6]:
#tokenize the after 2000 words
after2000TOK=nltk.word_tokenize(str(after2000lyr))

In [7]:
print(len(after2000TOK))
len(after2000TOK)/1849

819866


443.4104921579232

In [ ]:
after2000TOK

In [16]:
#remove stop words
stopwords=nltk.corpus.stopwords.words('english')
before2000stop = [w for w in before2000TOK if not w in stopwords]
after2000stop = [w for w in after2000TOK if not w in stopwords]

In [17]:
print(len(before2000stop))
print(len(before2000stop)/1951)
print(len(after2000stop))
print(len(after2000stop)/1849)

245458
125.81137878011276
382047
206.62358031368308


In [18]:
#see how affective lemmatization is
lemmatizer = WordNetLemmatizer()
before2000lem=[]
for i in before2000stop:
    before2000lem.append(lemmatizer.lemmatize(i))

In [19]:
after2000lem=[]
for i in after2000stop:
    after2000lem.append(lemmatizer.lemmatize(i))

In [20]:
print(len(before2000lem))
print(len(after2000lem))

245458
382047


In [ ]:
before2000stop

In [ ]:
before2000lem

In [23]:
songArtLyrics.to_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtLyrics.csv',index=False)

In [24]:
#now create frequency distributions. as seen below, the songs in before 2000s and after 2000s have very similar frequent words
beforeDist=FreqDist(before2000stop)
afterDist=FreqDist(after2000stop)

In [25]:
beforeDist

FreqDist({'love': 5334, 'know': 3742, 'want': 3659, 'baby': 3126, 'like': 2826, 'got': 2823, 'oh': 2399, "'": 2383, 'yeah': 2132, 'get': 2059, ...})

In [26]:
afterDist

FreqDist({'like': 6836, 'know': 5726, 'got': 5534, 'want': 5327, 'love': 4884, 'oh': 4268, 'get': 4228, 'baby': 3896, 'yeah': 3681, 'let': 3438, ...})

In [27]:
beforeDist_nitems=beforeDist.most_common(50)
afterDist_nitems=afterDist.most_common(50)

In [28]:
beforeDist_nitems

[('love', 5334),
 ('know', 3742),
 ('want', 3659),
 ('baby', 3126),
 ('like', 2826),
 ('got', 2823),
 ('oh', 2399),
 ("'", 2383),
 ('yeah', 2132),
 ('get', 2059),
 ('let', 2056),
 (',', 1950),
 ('one', 1930),
 ('time', 1921),
 ('go', 1858),
 ('make', 1810),
 ('going', 1789),
 ('say', 1697),
 ('see', 1607),
 ('come', 1551),
 ('never', 1546),
 ('would', 1473),
 ('way', 1444),
 ('feel', 1441),
 ('night', 1346),
 ('tell', 1299),
 ('girl', 1280),
 ('take', 1272),
 ('heart', 1245),
 ('right', 1195),
 ('need', 1141),
 ('could', 1141),
 ('give', 1101),
 ('us', 1061),
 ('back', 1029),
 ('life', 962),
 ('chorus', 961),
 ('away', 944),
 ('man', 838),
 ('think', 821),
 ('day', 818),
 ('little', 800),
 ('good', 797),
 ('keep', 790),
 ('around', 748),
 ('ever', 724),
 ('always', 721),
 ('world', 710),
 ('mind', 705),
 ('every', 688)]

In [29]:
afterDist_nitems

[('like', 6836),
 ('know', 5726),
 ('got', 5534),
 ('want', 5327),
 ('love', 4884),
 ('oh', 4268),
 ('get', 4228),
 ('baby', 3896),
 ('yeah', 3681),
 ('let', 3438),
 ('go', 3305),
 ('girl', 2671),
 ('make', 2545),
 ('see', 2519),
 ('going', 2513),
 ('one', 2494),
 ("'", 2398),
 ('say', 2316),
 ('never', 2179),
 ('back', 2081),
 ('time', 2068),
 ('take', 2035),
 ('way', 2016),
 ('right', 1901),
 ('come', 1899),
 (',', 1848),
 ('would', 1836),
 ('need', 1733),
 ('us', 1701),
 ('feel', 1634),
 ('tell', 1541),
 ('could', 1534),
 ('ya', 1436),
 ('life', 1383),
 ('give', 1366),
 ('night', 1339),
 ('keep', 1317),
 ('good', 1271),
 ('hey', 1263),
 ('think', 1237),
 ('man', 1233),
 ('la', 1111),
 ('ooh', 1072),
 ('away', 1067),
 ('heart', 1037),
 ('said', 1032),
 ('na', 1010),
 ('look', 1004),
 ('around', 987),
 ('boy', 986)]

In [30]:
#create basic brigrams below
before2000bi=list(nltk.bigrams(nltk.word_tokenize(str(before2000lyr))))
after2000bi=list(nltk.bigrams(nltk.word_tokenize(str(after2000lyr))))

In [31]:
before2000bi[:50]

[('[', "'colour"),
 ("'colour", 'me'),
 ('me', 'your'),
 ('your', 'colour'),
 ('colour', 'baby'),
 ('baby', 'colour'),
 ('colour', 'me'),
 ('me', 'your'),
 ('your', 'car'),
 ('car', 'colour'),
 ('colour', 'me'),
 ('me', 'your'),
 ('your', 'colour'),
 ('colour', 'darling'),
 ('darling', 'i'),
 ('i', 'know'),
 ('know', 'who'),
 ('who', 'you'),
 ('you', 'are'),
 ('are', 'come'),
 ('come', 'up'),
 ('up', 'off'),
 ('off', 'your'),
 ('your', 'colour'),
 ('colour', 'chart'),
 ('chart', 'i'),
 ('i', 'know'),
 ('know', 'where'),
 ('where', 'you'),
 ('you', 'are'),
 ('are', 'comin'),
 ('comin', 'from'),
 ('from', 'call'),
 ('call', 'me'),
 ('me', 'call'),
 ('call', 'me'),
 ('me', 'on'),
 ('on', 'the'),
 ('the', 'line'),
 ('line', 'call'),
 ('call', 'me'),
 ('me', 'call'),
 ('call', 'me'),
 ('me', 'any'),
 ('any', 'anytime'),
 ('anytime', 'call'),
 ('call', 'me'),
 ('me', 'call'),
 ('call', 'me'),
 ('me', 'my')]

In [32]:
after2000bi[:50]

[('[', "'"),
 ("'", 'i'),
 ('i', 'can'),
 ('can', 'feel'),
 ('feel', 'the'),
 ('the', 'magic'),
 ('magic', 'floating'),
 ('floating', 'in'),
 ('in', 'the'),
 ('the', 'air'),
 ('air', 'being'),
 ('being', 'with'),
 ('with', 'you'),
 ('you', 'gets'),
 ('gets', 'me'),
 ('me', 'that'),
 ('that', 'way'),
 ('way', 'i'),
 ('i', 'watch'),
 ('watch', 'the'),
 ('the', 'sunlight'),
 ('sunlight', 'dance'),
 ('dance', 'across'),
 ('across', 'your'),
 ('your', 'face'),
 ('face', 'and'),
 ('and', 'i'),
 ('i', 'have'),
 ('have', 'never'),
 ('never', 'been'),
 ('been', 'this'),
 ('this', 'swept'),
 ('swept', 'away'),
 ('away', 'my'),
 ('my', 'thoughts'),
 ('thoughts', 'just'),
 ('just', 'seem'),
 ('seem', 'to'),
 ('to', 'settle'),
 ('settle', 'on'),
 ('on', 'the'),
 ('the', 'breeze'),
 ('breeze', 'when'),
 ('when', 'i'),
 ('i', 'am'),
 ('am', 'lying'),
 ('lying', 'wrapped'),
 ('wrapped', 'up'),
 ('up', 'in'),
 ('in', 'your')]

In [33]:
#now create more measurable bigrams with collocations package
bigram_measures = nltk.collocations.BigramAssocMeasures()

before2000_finder=BigramCollocationFinder.from_words(nltk.word_tokenize(str(before2000lyr)))
after2000_finder=BigramCollocationFinder.from_words(nltk.word_tokenize(str(after2000lyr)))

before2000_scored=before2000_finder.score_ngrams(bigram_measures.raw_freq)
after2000_scored=after2000_finder.score_ngrams(bigram_measures.raw_freq)

In [34]:
before2000_scored[:50]

[(('i', 'am'), 0.007717450686813955),
 (('do', 'not'), 0.007077903553308957),
 (('it', 'is'), 0.0057335493747168184),
 (('you', 'are'), 0.004536496139410088),
 (('want', 'to'), 0.004394788902831721),
 (('i', 'will'), 0.0040535727937022325),
 (("'", ','), 0.0036359093595765193),
 (('can', 'not'), 0.0033487657486150914),
 (('and', 'i'), 0.0031455277382592757),
 (('in', 'the'), 0.002916185763270603),
 (('i', 'can'), 0.0028714361096142766),
 (('going', 'to'), 0.002703624908403053),
 (('i', 'want'), 0.002703624908403053),
 (('i', 'do'), 0.0026514169791373387),
 (('to', 'be'), 0.0025544593962152982),
 (('i', 'have'), 0.0023754607815899926),
 (('i', 'know'), 0.002112556566359075),
 (('you', 'know'), 0.00205475493038632),
 (('if', 'you'), 0.0019652556230736673),
 (('got', 'to'), 0.0019503390718548918),
 (('are', 'not'), 0.0019205059694173408),
 (('that', 'is'), 0.0018310066621046882),
 (('that', 'i'), 0.0017116742523544844),
 (('on', 'the'), 0.0017098096834521374),
 (('will', 'be'), 0.00169675

In [35]:
after2000_scored[:50]

[(('i', 'am'), 0.012517899266465496),
 (('do', 'not'), 0.007048712838439452),
 (('it', 'is'), 0.005008135475797265),
 (('you', 'are'), 0.0044019388534223885),
 (('want', 'to'), 0.0039701609775255955),
 (('in', 'the'), 0.0035835124276406144),
 (('and', 'i'), 0.003532284544059639),
 (('are', 'not'), 0.003144416282660825),
 (('can', 'not'), 0.0030224451312775504),
 (('i', 'will'), 0.0027797225400248335),
 (('i', 'do'), 0.0026638499462107224),
 (('i', 'can'), 0.002582129274783928),
 (("'", ','), 0.0022540268775629187),
 (('going', 'to'), 0.002169866783108459),
 (('i', 'know'), 0.0021344951492073094),
 (('i', 'want'), 0.0020417970741560204),
 (('you', 'know'), 0.001967394671812223),
 (('that', 'i'), 0.0019564172681877283),
 (('i', 'got'), 0.001908848519148251),
 (('to', 'be'), 0.0018710374622194358),
 (('on', 'the'), 0.0018149307325831293),
 (('if', 'you'), 0.0017466268878084956),
 (('oh', 'oh'), 0.0016880807351445236),
 (('but', 'i'), 0.0016295345824805516),
 (('to', 'the'), 0.001628314870

In [36]:
#remove stop words from both bigrams
before2000_finder.apply_word_filter(lambda w: w in stopwords)
after2000_finder.apply_word_filter(lambda w: w in stopwords)

before2000_scored=before2000_finder.score_ngrams(bigram_measures.raw_freq)
after2000_scored=after2000_finder.score_ngrams(bigram_measures.raw_freq)

In [37]:
before2000_scored[:50]

[(("'", ','), 0.0036359093595765193),
 (('yeah', 'yeah'), 0.0011895949596973433),
 (('let', 'us'), 0.0009788986737321397),
 (('oh', 'oh'), 0.0008260040237396913),
 ((',', "'"), 0.000805493765813875),
 (('na', 'na'), 0.0005351312749735697),
 (('oh', 'yeah'), 0.0003747783493717335),
 (('baby', 'baby'), 0.00034867438473887643),
 (('oh', 'baby'), 0.0003001955932778562),
 (('let', 'go'), 0.0002796853353520399),
 (('da', 'da'), 0.00027409162864499914),
 (('la', 'la'), 0.0002666333530356114),
 (('feel', 'like'), 0.00024612309510979515),
 (('really', 'want'), 0.00024052938840275434),
 (('little', 'bit'), 0.0002368002505980605),
 (('hey', 'hey'), 0.00021256085486755035),
 (('uh', 'uh'), 0.00020883171706285647),
 (('ooh', 'ooh'), 0.00020137344145346875),
 (('us', 'get'), 0.00018459232133234636),
 (('every', 'day'), 0.00018272775242999942),
 (('ah', 'ah'), 0.0001752694768206117),
 (('night', 'long'), 0.00017340490791826476),
 ((',', "'you"), 0.00016967577011357089),
 (('make', 'love'), 0.00016967

In [38]:
after2000_scored[:50]

[(("'", ','), 0.0022540268775629187),
 (('oh', 'oh'), 0.0016880807351445236),
 (('yeah', 'yeah'), 0.0011111571891016336),
 (('let', 'us'), 0.0011074980545601354),
 (('la', 'la'), 0.001041633632813167),
 (('na', 'na'), 0.0008562374827105893),
 ((',', "'"), 0.000668401909580346),
 (('hey', 'hey'), 0.0003817697038296502),
 (('feel', 'like'), 0.00036347403112215896),
 (('ooh', 'ooh'), 0.00029517018634752507),
 (('baby', 'baby'), 0.0002524802833633789),
 (('oh', 'yeah'), 0.0002451620142803824),
 (('every', 'time'), 0.00023906345671121867),
 (('ah', 'ah'), 0.00021588893794839646),
 (('da', 'da'), 0.00021588893794839646),
 (('uh', 'uh'), 0.00020369182281006895),
 (('us', 'get'), 0.00020369182281006895),
 (('us', 'go'), 0.00019515384221323973),
 (('really', 'want'), 0.00018783557313024324),
 (('whoa', 'whoa'), 0.0001768581695057485),
 (('low', 'low'), 0.000174418746478083),
 (('work', 'work'), 0.00017319903496425027),
 (('baby', 'girl'), 0.00016344134285358828),
 (('never', 'let'), 0.000159782

In [39]:
#apply a frequency filter of 5 times
before2000_finder_filt=BigramCollocationFinder.from_words(nltk.word_tokenize(str(before2000lyr)))
after2000_finder_filt=BigramCollocationFinder.from_words(nltk.word_tokenize(str(after2000lyr)))

before2000_finder_filt.apply_freq_filter(5)
after2000_finder_filt.apply_freq_filter(5)

#remove stopwords again
before2000_finder_filt.apply_word_filter(lambda w: w in stopwords)
after2000_finder_filt.apply_word_filter(lambda w: w in stopwords)

#remove words that are three letters or less
before2000_finder_filt.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
before2000_finder_filt.apply_ngram_filter(lambda w1, w2: len(w2) < 4)
after2000_finder_filt.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
after2000_finder_filt.apply_ngram_filter(lambda w1, w2: len(w2) < 4)

before2000_scored_filt=before2000_finder_filt.score_ngrams(bigram_measures.raw_freq)
after2000_scored_filt=after2000_finder_filt.score_ngrams(bigram_measures.raw_freq)

In [40]:
before2000_scored_filt[:50]

[(('yeah', 'yeah'), 0.0011895949596973433),
 (('baby', 'baby'), 0.00034867438473887643),
 (('feel', 'like'), 0.00024612309510979515),
 (('really', 'want'), 0.00024052938840275434),
 (('night', 'long'), 0.00017340490791826476),
 (('make', 'love'), 0.00016967577011357089),
 (('every', 'time'), 0.0001640820634065301),
 (('live', 'without'), 0.00013984266767601996),
 (('come', 'true'), 0.00013238439206663224),
 (('coming', 'back'), 0.0001305198231642853),
 (('turn', 'around'), 0.0001305198231642853),
 (('would', 'never'), 0.0001305198231642853),
 (('little', 'girl'), 0.00012306154755489757),
 (('repeat', 'chorus'), 0.00012306154755489757),
 (('come', 'back'), 0.0001193324097502037),
 (('love', 'shack'), 0.00011560327194550984),
 (('never', 'going'), 0.00011560327194550984),
 (('never', 'knew'), 0.00011560327194550984),
 (('every', 'night'), 0.0001137387030431629),
 (('real', 'love'), 0.00010814499633612211),
 (('love', 'like'), 0.00010441585853142824),
 (('would', 'like'), 0.00010255128962

In [41]:
after2000_scored_filt[:50]

[(('yeah', 'yeah'), 0.0011111571891016336),
 (('feel', 'like'), 0.00036347403112215896),
 (('baby', 'baby'), 0.0002524802833633789),
 (('every', 'time'), 0.00023906345671121867),
 (('really', 'want'), 0.00018783557313024324),
 (('whoa', 'whoa'), 0.0001768581695057485),
 (('work', 'work'), 0.00017319903496425027),
 (('baby', 'girl'), 0.00016344134285358828),
 (('dance', 'dance'), 0.0001341682665216023),
 (('shake', 'shake'), 0.00012563028592477306),
 (('night', 'long'), 0.00012441057441094032),
 (('would', 'never'), 0.00011953172835560934),
 (('goes', 'around'), 0.00011343317078644558),
 (('come', 'back'), 0.0001085543247311146),
 (('feels', 'like'), 0.0001085543247311146),
 (('love', 'love'), 0.00010733461321728184),
 (('turn', 'around'), 0.00010733461321728184),
 (('never', 'going'), 0.00010489519018961635),
 (('even', 'though'), 0.00010001634413428536),
 (('look', 'like'), 0.00010001634413428536),
 (('dance', 'floor'), 9.757692110661987e-05),
 (('walk', 'away'), 9.513749807895436e-05

In [42]:
#now compare PMI scores
before2000_finder_pmi=BigramCollocationFinder.from_words(nltk.word_tokenize(str(before2000lyr)))
after2000_finder_pmi=BigramCollocationFinder.from_words(nltk.word_tokenize(str(after2000lyr)))

before2000_finder_pmi.apply_freq_filter(5)
after2000_finder_pmi.apply_freq_filter(5)

#remove stopwords again
before2000_finder_pmi.apply_word_filter(lambda w: w in stopwords)
after2000_finder_pmi.apply_word_filter(lambda w: w in stopwords)

#remove words that are three letters or less
before2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
before2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w2) < 4)
after2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
after2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w2) < 4)

before2000_scored_pmi=before2000_finder_pmi.score_ngrams(bigram_measures.pmi)
after2000_scored_pmi=after2000_finder_pmi.score_ngrams(bigram_measures.pmi)

In [43]:
before2000_scored_pmi[:50]

[(('borrowed', 'tiiiime'), 16.710798363708996),
 (('domo', 'arigato'), 16.710798363708996),
 (('isla', 'bonita'), 16.710798363708996),
 (('quun', 'reve'), 16.710798363708996),
 (('fantasic', 'voyage'), 16.225371536538756),
 (('achy', 'breaky'), 16.032726458596358),
 (('coldhearted', 'snake'), 16.032726458596358),
 (('yaaaaaa', 'diditditditdit'), 16.032726458596358),
 (('source', 'companion'), 16.002979115202308),
 (('dodo', 'doda'), 15.862801457154045),
 (('marvin', 'gaye'), 15.862801457154045),
 (('wang', 'chung'), 15.862801457154045),
 (('opposites', 'attract'), 15.810334037259912),
 (('vida', 'loca'), 15.769692052762563),
 (('bette', 'davis'), 15.710798363708996),
 (('frustrated', 'incorporated'), 15.710798363708996),
 (('respresent', 'queens'), 15.710798363708996),
 (('coated', 'raindrops'), 15.670156379211651),
 (('beins', 'believin'), 15.573294839959061),
 (('sans', 'visage'), 15.573294839959061),
 (('yeux', 'sans'), 15.573294839959061),
 (('breakdown', 'takedown'), 15.4477639578

In [44]:
after2000_scored_pmi[:50]

[(('mardi', 'gras'), 17.32310051253267),
 (('babababadaba', 'bababadaba'), 17.060066106698876),
 (('cavalry', 'choirs'), 17.060066106698876),
 (('flinging', 'rawhide'), 17.060066106698876),
 (('haaaa', 'haaaaa'), 17.060066106698876),
 (('rahrahahahahah', 'romaromamamaa'), 17.060066106698876),
 (('romaromamamaa', 'gagaoohlala'), 17.060066106698876),
 (('woahohohohohohohohhh', 'woahohohohohohohohohhh'), 17.060066106698876),
 (('louisville', 'slugger'), 17.060066106698873),
 (('puerto', 'rican'), 17.060066106698873),
 (('roman', 'cavalry'), 16.83767368536243),
 (('andre', '3000'), 16.837673685362425),
 (('dopeman', 'dopeman'), 16.79703170086508),
 (('nobu', 'nobu'), 16.79703170086508),
 (('tgif', 'tgif'), 16.79703170086508),
 (('areumdawo', 'sarangseureowo'), 16.64502860742003),
 (('hateration', 'holleratin'), 16.64502860742003),
 (('nwod', 'gniht'), 16.64502860742003),
 (('pilf', 'nwod'), 16.64502860742003),
 (('powerglide', 'slime'), 16.64502860742003),
 (('whoaoaoh', 'beachin'), 16.645

In [45]:
#now compare PMI scores
before2000_finder_pmi=BigramCollocationFinder.from_words(nltk.word_tokenize(str(before2000lyr)))
after2000_finder_pmi=BigramCollocationFinder.from_words(nltk.word_tokenize(str(after2000lyr)))

before2000_finder_pmi.apply_freq_filter(10)
after2000_finder_pmi.apply_freq_filter(10)

#remove stopwords again
before2000_finder_pmi.apply_word_filter(lambda w: w in stopwords)
after2000_finder_pmi.apply_word_filter(lambda w: w in stopwords)

#remove words that are three letters or less
before2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
before2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w2) < 4)
after2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w1) < 4)
after2000_finder_pmi.apply_ngram_filter(lambda w1, w2: len(w2) < 4)

before2000_scored_pmi=before2000_finder_pmi.score_ngrams(bigram_measures.pmi)
after2000_scored_pmi=after2000_finder_pmi.score_ngrams(bigram_measures.pmi)

In [46]:
before2000_scored_pmi[:50]

[(('raspberry', 'beret'), 15.225371536538756),
 (('licky', 'boomboom'), 15.109894319118819),
 (('billie', 'jean'), 14.94526361734602),
 (('tusk', 'tusk'), 14.85780077609568),
 (('sexual', 'healing'), 14.682229211512224),
 (('suckle', 'chocolate'), 14.659268063068913),
 (('lyrical', 'gangster'), 14.640409035817598),
 (('tuesday', 'wednesday'), 14.52176453931898),
 (('chaka', 'khan'), 14.447763957875202),
 (('total', 'eclipse'), 14.371660978789409),
 (('chocolate', 'dripping'), 14.36976144587393),
 (('wednesday', 'thursday'), 14.277838956432891),
 (('kris', 'kross'), 14.225371536538756),
 (('simply', 'irresistible'), 14.216809523035332),
 (('monday', 'tuesday'), 14.199836444431618),
 (('pink', 'cadillac'), 14.085193878490495),
 (('humpty', 'hump'), 14.03272645859636),
 (('dazzey', 'duks'), 13.945263617346018),
 (('master', 'blaster'), 13.907195576512498),
 (('nahah', 'nahah'), 13.892493142090851),
 (('pacman', 'fever'), 13.784798945152774),
 (('choo', 'choo'), 13.710798363708994),
 (('oh

In [47]:
after2000_scored_pmi[:50]

[(('heeeyyy', 'yaaaaaaa'), 16.060066106698876),
 (('heeyy', 'yaaaaaaaa'), 16.060066106698876),
 (('stutters', 'stastutter'), 16.060066106698873),
 (('ahora', 'vengo'), 15.837673685362429),
 (('oofisoyloimeyluvinoywutchasnow', 'oofisoyloimeyluvin'),
  15.837673685362425),
 (('eenie', 'meenie'), 15.64502860742003),
 (('elei', 'eleeh'), 15.64502860742003),
 (('meenie', 'miney'), 15.64502860742003),
 (('butterfly', 'sugarbaby'), 15.557565766169692),
 (('marvin', 'gaye'), 15.475103605977719),
 (('copyright', 'lyrics'), 15.397101093976445),
 (('dismissed', 'dismissed'), 15.229991108141189),
 (('fikir', 'fikir'), 15.229991108141189),
 (('ongo', 'getcho'), 15.212069200143926),
 (('sexual', 'eruption'), 15.10759447678146),
 (('badum', 'bood'), 14.890141105256564),
 (('bailando', 'bailando'), 14.890141105256564),
 (('stastutter', 'stutter'), 14.890141105256564),
 (('motherfucking', 'starboy'), 14.890141105256562),
 (('stutter', 'stutters'), 14.89014110525656),
 (('dabadee', 'dabadie'), 14.837673